In [182]:
import pandas as pd
import os

#os.listdir()
df = pd.read_csv('train_revised.csv')
df.head(2)

,ride_id,seat_number,payment_method,payment_receipt,travel_date,travel_time,travel_from,travel_to,car_type,max_capacity
0,1442,15A,Mpesa,UZUEHCBUSO,17-10-17,7:15,Migori,Nairobi,Bus,49
1,5437,14A,Mpesa,TIHLBUSGTE,19-11-17,7:12,Migori,Nairobi,Bus,49


In [196]:
def mode(x):
    from collections import Counter
    F = Counter(x)
    F = F.most_common(1)[0][0]
    
    return F
X = pd.DataFrame()
Df = df.groupby('ride_id')
Y = Df.max_capacity.agg('count')
X['payment_method'] = Df.payment_method.agg(mode)
X['travel_date'] = Df.travel_date.agg(mode)
X['travel_from'] = Df.travel_from.agg(mode)
X['travel_to'] = Df.travel_to.agg(mode)
X['car_type'] = Df.car_type.agg(mode)
X['max_capacity'] = Df.max_capacity.agg(mode)

Y.head(1)
print(X.shape,Y.shape)

X.head(1)
X.travel_date = pd.DatetimeIndex(X.travel_date)
X['Day'] = [i.day for i in X.travel_date]
X['Day_of_week'] = [i.dayofweek for i in X.travel_date ]
X['is_weekend'] = [1 if (i == 5 or i == 6) else 0 for i in X['Day_of_week']]
X['Month'] = [i.month for i in X.travel_date]
X['Year'] = [i.year for i in X.travel_date]
# Create Dummies For the cols
Cat = ['travel_from','car_type','Year','payment_method']
Good = ['Day_of_week','Month','is_weekend']
X_clean = X[Good]
for i in Cat:
    col = X[i]
    Dum = pd.get_dummies(col)
    X_fin = pd.concat((X_clean,Dum),1)
    X_clean = X_fin
print(X_fin.shape,Y.shape)
X_fin.head(2)
X_fin['Y'] = Y

(6249, 6) (6249,)
(6249, 26) (6249,)


In [177]:
'''# Test the Correlations
#X_fin.corr(method  = 'pearson')
from statsmodels.stats.outliers_influence import variance_inflation_factor    

def calculate_vif_(X, thresh=5.0):
    variables = list(range(X.shape[1]))
    dropped = True
    while dropped:
        dropped = False
        vif = [variance_inflation_factor(X.iloc[:, variables].values, ix)
               for ix in range(X.iloc[:, variables].shape[1])]

        maxloc = vif.index(max(vif))
        if max(vif) > thresh:
            print('dropping \'' + X.iloc[:, variables].columns[maxloc] +
                  '\' at index: ' + str(maxloc))
            del variables[maxloc]
            dropped = True

    print('Remaining variables:')
    print(X.columns[variables])
    return X.iloc[:, variables]
X_fin = calculate_vif_(X_fin)
X_fin.shape
'''

"# Test the Correlations\n#X_fin.corr(method  = 'pearson')\nfrom statsmodels.stats.outliers_influence import variance_inflation_factor    \n\ndef calculate_vif_(X, thresh=5.0):\n    variables = list(range(X.shape[1]))\n    dropped = True\n    while dropped:\n        dropped = False\n        vif = [variance_inflation_factor(X.iloc[:, variables].values, ix)\n               for ix in range(X.iloc[:, variables].shape[1])]\n\n        maxloc = vif.index(max(vif))\n        if max(vif) > thresh:\n            print('dropping '' + X.iloc[:, variables].columns[maxloc] +\n                  '' at index: ' + str(maxloc))\n            del variables[maxloc]\n            dropped = True\n\n    print('Remaining variables:')\n    print(X.columns[variables])\n    return X.iloc[:, variables]\nX_fin = calculate_vif_(X_fin)\nX_fin.shape\n"

## Baseline Model Without Uber Data

In [197]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler as scale


x_train,x_test,y_train,y_test = train_test_split(X_fin,Y,test_size = 0.1)

SS = scale()
SS.fit(x_train)
G = SS.transform(x_train)

RF = RandomForestRegressor(verbose = 1,n_estimators = 100)
RF.fit(G,y_train)

Pred = RF.predict(x_test)



/home/kamau/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/kamau/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  # This is added back by InteractiveShellApp.init_path()
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


In [198]:
print(mae(y_test,Pred))


30.664688


## Testing The Model on the Test Set

In [201]:
test =  pd.read_csv('test_questions.csv')
X = test

X.travel_date = pd.DatetimeIndex(X.travel_date)
X['Day'] = [i.day for i in X.travel_date]
X['Day_of_week'] = [i.dayofweek for i in X.travel_date ]
X['is_weekend'] = [True if (i == 5 or i == 6) else False for i in X['Day_of_week']]
X['Month'] = [i.month for i in X.travel_date]
X['Year'] = [i.year for i in X.travel_date]

Cat = ['travel_from','car_type','Year']
Good = ['Day_of_week','Month','is_weekend']
X_test_clean = X[Good]
for i in Cat:
    col = X[i]
    Dum = pd.get_dummies(col)
    X_fins = pd.concat((X_test_clean,Dum),1)
    X_test_clean = X_fins
print(X_fins.shape,X_fin.shape)
nots = []
for i in X_fin.columns:
    if not i in X_fins.columns:
        nots.append(i)
    else:
        pass
for i in nots:
    X_fins[i] = 0
'''noted = []
for i in X_fins.columns:
    if not i in X_fin.columns:
        noted.append(i)
    else:
        pass
X_fins = X_fins[X_fins.columns.difference(noted)]'''
print(X_fins.shape,X_fin.shape)
import numpy as np

Pred_test = RF.predict(X_fins)
#P = pd.DataFrame(Pred_test).apply(np.ceiling)
P = pd.DataFrame(Pred_test.round())

G = pd.concat((test.ride_id,P),1)
G.head()
G.to_csv('Baseline_model.csv',index = False)


(1111, 21) (6249, 27)
(1111, 27) (6249, 27)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


In [193]:
nots = []
for i in X_fins.columns:
    if not i in X_fin.columns:
        nots.append(i)
    else:
        pass
nots

[False, True]